# Simple XTA Simulation Example: field1.dcm

## Simulation Setup

In [14]:
%matplotlib widget
import os, sys

sys.path.append('..')
from xta import xta_sim

#### Setting Astra Path

In [15]:
#Possible generalization for environment dependency path
os.environ['ASTRA_BIN'] = os.getenv('ASTRA_BIN')
#print(os.getenv('ASTRA_BIN'))

In [16]:
#Edit this line to match your path if the above cell doesn't work
#%env ASTRA_BIN=/Users/colinrecker/Desktop/SLACForks/Astra

#### Setting Simulation Directory Path

In [17]:
#Define xta repo path
xta_path = '/Users/colinrecker/Desktop/SLACForks/xta'
os.chdir(xta_path)

#Define simulations directory path
sim_path = '/Users/colinrecker/Google Drive/My Drive/Colin/Simulations'

#Defining tumor scan path and scrape name of file
init_dcm_path = os.path.join(sim_path, 'inputs', 'Fluence Maps Anon', 'Field 3 Anon', 'field3.dcm')

#Defining vcc laser images path
vcc_path = os.path.join(sim_path, 'inputs', 'laser_mat_images', '*.mat')

#Creating xta_sim object
field1 = xta_sim(xta_path, sim_path, init_dcm_path, vcc_path)

AttributeError: 'xta_sim' object has no attribute 'IMAGE'

### Generating Initial Electron Distribution

In [ ]:
#Configuring distribution settings (distgen.yaml file)
field1.dist.input['n_particle'] = int(1e4)
fwhm = 0.06/2.355 # 60 fs = 0.06 ps
field1.dist.input['t_dist']['sigma_t']['value'] = fwhm

#Generating distribution with first index of laser images
particles, dist = field1.generate_dist(0)

## Run Simulation

### Configure Astra

In [ ]:
#Adjusting astra inputs (xta.in file)
field1.astra.input['newrun']['zstart']=0.0
field1.astra.input['newrun']['zstop'] = 0.25
field1.astra.input['charge']['lspch'] = True
field1.astra.input['newrun']['zphase'] = 50
field1.astra.input['solenoid']['maxb(1)'] = 0.427

### Run Single Simulation

In [28]:
field1.simulation('single')

#Archive simulation with desired .h5 file name
field1.archive(f'B={field1.astra.input["solenoid"]["maxb(1)"]}')

## Preset Plots

### Single Plots

#### Laser Image Plot

In [ ]:
field1.plot.single_dist('laser_image')

#### Initial Electron Distribution

In [ ]:
field1.plot.single('initial_dist', xlim = [-10, 10], ylim = [-10, 10], title = 'Initial Distribution', save=False, close=True)

#### Final Electron Distribution

In [ ]:
field1.plot.single('final_dist')

#### Rotated Final Distribution

In [ ]:
field1.plot.single('rotated_final_dist')

### Compare Plots

#### Initial vs. Final Distribution

In [ ]:
field1.plot.compare('initial_vs_final')

#### Initial vs. Rotated Final Distribution

In [ ]:
field1.plot.compare('inital_vs_rotated')

### Movie Plot

In [ ]:
field1.movie()

## Custom Plots

In [ ]:
xtapart = field1.astra_particles
